In [ ]:
!pip install openai
import pandas as pd
import os
import openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 12.3 MB/s 
  Created wheel for openai: filename=openai-0.19.0-py3-none-any.whl size=53535 sha256=26f7d5296e376dec61c4bc1b9aa4e9fc38f4fff6fd0b52c6b6519d752a279a62
  Stored in directory: /root/.cache/pip/wheels/94/b5/c0/928013bd6418b23b9c5d89fb24cdeb1faae899c11377d69609
Successfully built openai


In [ ]:
mdf = pd.read_excel('./Dataset/FeasibilityQA_dataset_MCQ.xlsx')
bdf = pd.read_excel('./Dataset/FeasibilityQA_dataset_BCQ.xlsx')

In [ ]:
bdf.head()

,category,subcategory,knowledge,premise,hypothesis,binary_classification_label,author
0,attribute_comparison,account_balance,One can not withdraw more money than the amoun...,Charlie has 622 dollars in his account. He wan...,The cost of the vehicle is 500. Charlie could ...,True,Neeraj
1,attribute_comparison,account_balance,One can not withdraw more money than the amoun...,Charlie has 622 dollars in his account. He wan...,The cost of the vehicle is 800. Charlie could ...,False,Neeraj
2,attribute_comparison,account_balance,One can not withdraw more money than the amoun...,Jackson wants to buy a sports car. He has 534 ...,The cost of the vehicle is 500. Jackson could ...,True,Neeraj
3,attribute_comparison,account_balance,One can not withdraw more money than the amoun...,Jackson wants to buy a sports car. He has 534 ...,The cost of the vehicle is 700. Jackson could ...,False,Neeraj
4,attribute_comparison,account_balance,One can not withdraw more money than the amoun...,Cost of a bike is 692 dollars. William wants t...,William has 700 dollars in his account. Willia...,True,Neeraj


In [ ]:
mdf.head()

,category,subcategory,premise,knowledge,question,question_text,options,value_answer,option_answer,author
0,attribute_comparison,account_balance,Charlie has 622 dollars in his account. He wan...,One can not withdraw more money than the amoun...,{'question_text': 'At what cost could Charlie ...,At what cost could Charlie buy the vehicle fro...,"[{'A': 550}, {'B': 350}, {'C': 650}, {'D': 750...","[550, 350]","['A', 'B']",Neeraj
1,attribute_comparison,account_balance,Jackson wants to buy a sports car. He has 534 ...,One can not withdraw more money than the amoun...,{'question_text': 'At what cost could Jackson ...,At what cost could Jackson buy the vehicle fro...,"[{'A': 550}, {'B': 400}, {'C': 500}, {'D': 650}]","[400, 500]","['B', 'C']",Neeraj
2,attribute_comparison,account_balance,Cost of a bike is 692 dollars. William wants t...,One can not withdraw more money than the amoun...,{'question_text': 'How much money William coul...,How much money William could have to buy the v...,"[{'A': 500}, {'B': 600}, {'C': 800}, {'D': 450...",[800],['C'],Neeraj
3,attribute_comparison,account_balance,Cost of a bike is 552 dollars. Nicolas wants t...,One can not withdraw more money than the amoun...,{'question_text': 'How much money Nicolas coul...,How much money Nicolas could have to buy the v...,"[{'A': 750}, {'B': 500}, {'C': 450}, {'D': 550...",[750],['A'],Neeraj
4,attribute_comparison,age,Cole is 72 years old.,Son's age would be less than that of the father,"{'question_text': ""How old could Cole's son be...",How old could Cole's son be?,"[{'A': 47}, {'B': 22}, {'C': 97}, {'D': 72}]","[47, 22]","['A', 'B']",Neeraj


In [ ]:
#Question for binary question Zero Shot
question_binary = 'Validate the hypothesis as True or False based on Knowledge and Premise:'

#Learning samples for binary questions One & Few Shot
examples_list_binary = [
                    {'knowledge': "One can not withdraw more money than the amount in their bank account.",
                    'premise': "Charlie has 622 dollars in his account. He wants to buy a bike. The cost of the vehicle is 500.",
                    'hypothesis': "Charlie could buy the vehicle from his account", 
                    'result': True
                    }, 
                    {'knowledge': "Resistance is inversely proportional to the area of the metalic wire.",
                    'premise': "Victor's experiment is to observe the relationship between resistance and area using two similar wires of different areas. The area of the wire with larger resistance is 32 units.",
                    'hypothesis': "Area of the other wire could be 25 units.", 
                    'result': False
                    }, 
                    {'knowledge': "Legal age to vote is 18 years in India.",
                    'premise': "Evan is a citizen of India and he recently cast his first vote in the national election.",
                    'hypothesis': "Evan is 21 years old when he casted the vote.", 
                    'result': True
                    }, 
                    {'knowledge': "To pass an exam, you need to get at least passing marks.",
                    'premise': "Anthony passed an exam where the passing score was 77.",
                    'hypothesis': "Anthony could have scored 76 marks in that exam.", 
                    'result': False
                    }
                ]

#Question for MCQ questions Zero Shot
question_mcq = 'Choose the correct option(s) based on Knowledge (if given) and Premise:'

#Learning samples for MCQ questions One & Few Shot
examples_list_mcq = [
                    {'knowledge': "One can not withdraw more money than the amount in their bank account.",
                    'premise': "Charlie has 622 dollars in his account. He wants to buy a bike. The cost of the vehicle is 500.",
                    'question': "At what cost could Charlie buy the vehicle from his account?",
                     'options': [{'A': 550}, {'B': 350}, {'C': 650}, {'D': 750}, {'E': 'None'}], 
                    'result': [550, 350]
                    }, 
                    {'knowledge': "Smaller planet has a smaller radius in comparison to a larger planet.",
                    'premise': "Planet A is bigger in radius that planet B. The radius of planet A is 43 units.",
                    'question': "What could be the radius of planet B?",
                     'options': [{'A': 46}, {'B': 39}, {'C': 43}, {'D': 41}],
                    'result': [39, 41]
                    }, 
                    {'knowledge': "Potential energy is directly proportional to the height.",
                    'premise': "Antonio's experiment is to observe the relationship between potential energy and height of two identical balls. The potential energy of the ball at lower height is 29 units.",
                    'question': "What could be the potential energy of the other ball?",
                     'options': [{'A': 31}, {'B': 28}, {'C': 33}, {'D': 24}], 
                    'result': [31, 33]
                    }, 
                    {'knowledge': "Speed of the upstream river reduces the speed of the raft.",
                    'premise': "Samuel participates in a boat racing competition. He first rows the boat in still water with a speed of 20 and then starts to go upstream.",
                    'question': "What could be the overall speed of the raft in upstream?",
                     'options': [{'A': 24}, {'B': 15}, {'C': 16}, {'D': 19}], 
                    'result': [15, 16, 19]
                    }
                ]

In [ ]:
#Get prompts in required format
def create_prompts(val, question_type, examples = None, question_zs = None, knowledge = True, premise = True, shot = 'zero'):
    if shot == 'zero':
        prompt_str = question_zs + "\n"
        if knowledge:
            knowledge_var = f"Knowledge: {val['knowledge']}\n"
            prompt_str += knowledge_var 
        if premise:
            premise_var = f"Premise: {val['premise']}\n"
            prompt_str += premise_var

        if question_type == 'binary':
            prompt_str += f"Hypothesis: {val['hypothesis']}\nResult:"
        if question_type == 'mcq':
            prompt_str += f"Question: {val['question_text']}\n"
            prompt_str += f"Options: {val['options']}\nResult:"

        return prompt_str

    elif shot == 'one':
        try:
            prompt_str = ''
            for item, text in examples[0].items():
                if knowledge and item  == 'knowledge':
                    knowledge_var = f"Knowledge: {text}\n"
                    prompt_str += knowledge_var 
                if premise and item  == 'premise':
                    premise_var = f"Premise: {text}\n"
                    prompt_str += premise_var
                if item == 'hypothesis':
                    prompt_str += f"Hypothesis: {text}\n"
                if item == 'question':
                    prompt_str += f"Question: {text}\n"
                if item == 'options':
                    prompt_str += f"Options: {text}\n"
                if item == 'result': 
                    prompt_str += f"Result: {text}\n\n" 
        except:
            raise Exception("Provide atleast one example")

        if knowledge:
            knowledge_var = f"Knowledge: {val['knowledge']}\n"
            prompt_str += knowledge_var 
        if premise:
            premise_var = f"Premise: {val['premise']}\n"
            prompt_str += premise_var
        
        if question_type == 'binary':
            prompt_str += f"Hypothesis: {val['hypothesis']}\nResult:"
        if question_type == 'mcq':
            prompt_str += f"Question: {val['question_text']}\n"
            prompt_str += f"Hypothesis: {val['options']}\nResult:"
        return prompt_str
    
    elif shot == 'few':
        try:
            prompt_str = ''
            for example in examples:
                for item, text in example.items():
                    if knowledge and item  == 'knowledge':
                        knowledge_var = f"Knowledge: {text}\n"
                        prompt_str += knowledge_var 
                    if premise and item  == 'premise':
                        premise_var = f"Premise: {text}\n"
                        prompt_str += premise_var
                    if item == 'hypothesis':
                        prompt_str += f"Hypothesis: {text}\n"
                    if item == 'question':
                        prompt_str += f"Question: {text}\n"
                    if item == 'options':
                        prompt_str += f"Options: {text}\n"
                    if item == 'result': 
                        prompt_str += f"Result: {text}\n\n" 
        except:
            raise Exception("Provide atleast one example")

        if knowledge:
            knowledge_var = f"Knowledge: {val['knowledge']}\n"
            prompt_str += knowledge_var 
        if premise:
            premise_var = f"Premise: {val['premise']}\n"
            prompt_str += premise_var
        
        if question_type == 'binary':
            prompt_str += f"Hypothesis: {val['hypothesis']}\nResult:"
        if question_type == 'mcq':
            prompt_str += f"Question: {val['question_text']}\n"
            prompt_str += f"Hypothesis: {val['options']}\nResult:"
        return prompt_str
    
    else:
        raise Exception("Not a valid entry for shot argument. It can take values which are: zero, one or few")


#Get response from GPT3
def gpt3_response(df, output_path, pred_col_name = 'GPT3Pred', start_idx = 0, end_idx = None, knowledge = True, premise = True, shot = 'one', 
                  engine = 'text-davinci-002', temperature = 0.5, max_tokens = 256, top_p = 1, frequency_penalty = 0, presence_penalty = 0, dump = True, 
                  question_type = 'binary', sample_size = None):
    
    print('Data size: ', df.shape[0])
    
    if isinstance(sample_size, float):
        df = df.sample(frac = sample_size).reset_index().reset_index(drop = True)
        print('Data size after resampling: ', df.shape[0])
    elif isinstance(sample_size, int):
        df = df.sample(n = sample_size).reset_index(drop = True)
        print('Data size after resampling: ', df.shape[0])
    else:
        pass
    
    #If new file create new column
    if start_idx == 0:
        df[pred_col_name] = None

    if end_idx is None:
        end_idx = len(df)
    
    for idx, row in enumerate(df.iloc[start_idx:end_idx].iterrows(), start_idx):
        print('Row Number: ', idx+1)
        if question_type == 'binary':
            prompt_str = create_prompts(row[1], question_type, examples = examples_list_binary, question_zs = question_binary, 
                                        knowledge = knowledge, premise = premise, shot = shot)
        if question_type == 'mcq':
            prompt_str = create_prompts(row[1], question_type, examples = examples_list_mcq, question_zs = question_mcq, 
                                        knowledge = knowledge, premise = premise, shot = shot)
        try:
            response = openai.Completion.create(
                                engine=engine,
                                prompt= prompt_str,
                                temperature=temperature,
                                max_tokens=max_tokens,
                                top_p=top_p,
                                frequency_penalty=frequency_penalty,
                                presence_penalty=presence_penalty
                            )
            result = response['choices'][0]['text'].strip()
            print(prompt_str + ' ' + result)
            df.loc[idx, pred_col_name] = result
            print('\n**********************\n')
            
            if dump:
                df.to_csv(output_path, index = False)
        except:
            print('Resources exhausted probably!!!')
            break

    if not dump:
        return df


#Performance - Metrics
def metrics(df, output_path, pred_col_name = 'GPT3Pred', dump = True, metrics = ['accuracy'], question_type = 'binary', shot = None):

    #Correct anomalies in GPT 3 responses
    if question_type == 'binary':
        try:
            df[pred_col_name] = df[pred_col_name].apply(lambda x: True if 'true' in x.lower() else False)
        except AttributeError:
            pass
    
    if question_type == 'mcq':
        df['value_answer'] = df['value_answer'].apply(lambda x: x.replace('[', '').replace(']', '').split(', '))
        df['option_answer'] = df['option_answer'].apply(lambda x: x.replace('[', '').replace(']', '').replace("'", '').split(', '))
        df['TotalRes'] = df['value_answer'].apply(lambda x: len(x))

        def get_tp(ans_list, opt_list, preds):
            cnt = 0
            for ans, opt in zip(ans_list, opt_list):
                if ans.replace("'", '') in preds or opt.replace("'", '') in preds:
                    cnt+=1
            return cnt

        def get_fp(opt_name_list, opt_val_list, preds):
            cnt = 0
            tot = len(opt_name_list)
            for opt_name, opt_val in zip(opt_name_list, opt_val_list):
                if opt_name in preds or opt_val in preds:
                    cnt+=1
            return tot - cnt

        df['opt_names'] = df['options'].apply(lambda x: [i.split(': ')[0] for i in x.replace('[', '').replace(']', '').replace("'", '').replace("{", '').replace("}", '').split(', ')])
        df['opt_vals'] = df['options'].apply(lambda x: [i.split(': ')[-1] for i in x.replace('[', '').replace(']', '').replace("'", '').replace("{", '').replace("}", '').split(', ')])

        df['TruePositives'] = df[['value_answer', 'option_answer', pred_col_name]].apply(lambda x: get_tp(x[0], x[1], x[2]), axis=1)
        df['TrueNegatives'] = df[['opt_names', 'opt_vals', pred_col_name]].apply(lambda x: get_fp(x[0], x[1], x[2]), axis=1)

        df.drop(['opt_names', 'opt_vals'], axis = 1, inplace = True)

        if shot in ['one', 'few']:
            df['GPT3Pred2List'] = df['GPT3Pred'].apply(lambda x: x.replace('[', '').replace(']', '').replace('{', '').replace('}', '').split(', '))
            df['GPT3Pred2List_Len'] = df['GPT3Pred2List'].apply(lambda x: len(x))
            df['FalseNegatives'] = df['GPT3Pred2List_Len'] - df['TotalRes']
            df['FalsePositives'] = df['FalseNegatives'].apply(lambda x: abs(x) if x < 0 else 0)
            df['FalseNegatives'] = df['FalseNegatives'].apply(lambda x: 0 if x < 0 else 0)
            

    if dump:
        df.to_csv(output_path, index = False)

    for metric in metrics:
        if metric == 'accuracy' and question_type == 'binary':
            print('Accuracy: ', round(df[['binary_classification_label', pred_col_name]].apply(lambda x: x[0] == x[1], axis=1).sum()*100/len(df), 4), '%')
        if metric == 'accuracy' and question_type == 'mcq':
            print('Accuracy: ', round(df['TruePositives'].sum()*100/df['TotalRes'].sum(), 2), '%')
            print('Accuracy2: ', round(df['TruePositives'].sum()*100/(df['TruePositives'].sum() + df['TrueNegatives'].sum() \
                                                                     + df['FalsePositives'].sum() + df['FalseNegatives'].sum()), 2), '%')
            if shot in ['one', 'few']:
                print('FPR: ', round(df['FalsePositives'].sum()*100/df['TotalRes'].sum(), 2), '%')

    if not dump:
        return df

# Inputs, API Token and Examples

In [ ]:
openai.api_key = "ADD TOKEN"

In [ ]:
#Binary
#Zero Shot Prompt - Binary
print('****Zero Shot Prompt Generation****')
print(create_prompts(bdf.iloc[0], question_zs = question_binary, knowledge = True, premise = True, shot = 'zero'))

#One Shot Prompt - Binary
print('\n****One Shot Prompt Generation****')
print(create_prompts(bdf.iloc[0], examples = examples_list_binary, knowledge = True, premise = True, shot = 'one'))

#Few Shot Prompt - Binary
print('\n****Few Shot Prompt Generation****')
print(create_prompts(bdf.iloc[0], examples = examples_list_binary, knowledge = True, premise = True, shot = 'few'))

****Zero Shot Prompt Generation****
Validate the hypothesis as True or False based on Knowledge and Premise:
Knowledge: One can not withdraw more money than the amount in their bank account
Premise: Charlie has 622 dollars in his account. He wants to buy a bike.
Hypothesis: The cost of the vehicle is 500. Charlie could buy the vehicle from his account.
Result:

****One Shot Prompt Generation****
Knowledge: One can not withdraw more money than the amount in their bank account.
Premise: Charlie has 622 dollars in his account. He wants to buy a bike. The cost of the vehicle is 500.
Hypothesis: Charlie could buy the vehicle from his account
Result: True

Knowledge: One can not withdraw more money than the amount in their bank account
Premise: Charlie has 622 dollars in his account. He wants to buy a bike.
Hypothesis: The cost of the vehicle is 500. Charlie could buy the vehicle from his account.
Result:

****Few Shot Prompt Generation****
Knowledge: One can not withdraw more money than the

In [ ]:
#MCQ
#Zero Shot Prompt - Binary
print('****Zero Shot Prompt Generation****')
print(create_prompts(mdf.iloc[0], question_zs = question_mcq, knowledge = True, premise = True, shot = 'zero', question_type = 'mcq'))

#One Shot Prompt - Binary
print('\n****One Shot Prompt Generation****')
print(create_prompts(mdf.iloc[0], examples = examples_list_mcq, knowledge = True, premise = True, shot = 'one', question_type = 'mcq'))

#Few Shot Prompt - Binary
print('\n****Few Shot Prompt Generation****')
print(create_prompts(mdf.iloc[0], examples = examples_list_mcq, knowledge = True, premise = True, shot = 'few', question_type = 'mcq'))

****Zero Shot Prompt Generation****
Choose the correct option based on Knowledge (if given) and Premise:
Knowledge: One can not withdraw more money than the amount in their bank account
Premise: Charlie has 622 dollars in his account. He wants to buy a bike.
Options: [{'A': 550}, {'B': 350}, {'C': 650}, {'D': 750}, {'E': 'None'}]
Result:

****One Shot Prompt Generation****
Knowledge: One can not withdraw more money than the amount in their bank account.
Premise: Charlie has 622 dollars in his account. He wants to buy a bike. The cost of the vehicle is 500.
Question: At what cost could Charlie buy the vehicle from his account?
Options: [{'A': 550}, {'B': 350}, {'C': 650}, {'D': 750}, {'E': 'None'}]
Result: [550, 350]

Knowledge: One can not withdraw more money than the amount in their bank account
Premise: Charlie has 622 dollars in his account. He wants to buy a bike.
Question: At what cost could Charlie buy the vehicle from his account?
Hypothesis: [{'A': 550}, {'B': 350}, {'C': 650},

# Binary Question Type

## Zero Shot Validation - With Knowledge

In [ ]:
#Zero Shot Results - Without Knowledge
outpath = '/content/drive/MyDrive/feasibilityQA/Binary/ZeroShot_WK_Output.csv'
corr_outpath = '/content/drive/MyDrive/feasibilityQA/Binary/ZeroShot_WK_Output_Corrected.csv'

zsdf1 = bdf.copy()
gpt3_response(zsdf1, outpath, knowledge = True, shot = 'zero', dump = True)

Streaming output truncated to the last 5000 lines.
Hypothesis: The current month could be February.
Result: True

**********************

Row Number:  2694
Validate the hypothesis as True or False based on Knowledge and Premise:
Knowledge: February is the shortest month of the year.
Premise: Cole's birthday comes in the current month. It is the shortest month out of all the twelve months.
Hypothesis: The current month could be March.
Result: False

**********************

Row Number:  2695
Validate the hypothesis as True or False based on Knowledge and Premise:
Knowledge: February is the shortest month of the year.
Premise: Maxwell was short on cash and was eagerly waiting for his salary. He was relived because the current month was the shortest month of the year.
Hypothesis: The current month could be February.
Result: True

**********************

Row Number:  2696
Validate the hypothesis as True or False based on Knowledge and Premise:
Knowledge: February is the shortest month of th

In [ ]:
#Performance - Metrics
zsdf1 = pd.read_csv(outpath)
metrics(zsdf1, corr_outpath, dump = True)

Accuracy:  69.33 %


## Zero Shot Validation - Without Knowledge

In [ ]:
#Zero Shot Results - Without Knowledge
outpath = '/content/drive/MyDrive/feasibilityQA/Binary/ZeroShot_WOK_Output.csv'
corr_outpath = '/content/drive/MyDrive/feasibilityQA/Binary/ZeroShot_WOK_Output_Corrected.csv'

zsdf2 = bdf.copy()
gpt3_response(zsdf2, outpath, knowledge = False, shot = 'zero', dump = True)

Streaming output truncated to the last 5000 lines.
Row Number:  2624
Validate the hypothesis as True or False based on Knowledge and Premise:
Premise: Jayden went to a supermarket and purchased something worth 3 dollars, he paid the full amount in denomenation of one cent 
Hypothesis: He could have given at most 500 coins of 1 cent.
Result: True

**********************

Row Number:  2625
Validate the hypothesis as True or False based on Knowledge and Premise:
Premise: Jonah went for a blood test,and got high cholesterol results.
Hypothesis: It could have been because of eating excessive fast food.
Result: False

**********************

Row Number:  2626
Validate the hypothesis as True or False based on Knowledge and Premise:
Premise: Jonah went for a blood test,and got high cholesterol results.
Hypothesis: It could have been because of doing excessive exercises.
Result: False

**********************

Row Number:  2627
Validate the hypothesis as True or False based on Knowledge and Prem

In [ ]:
#Performance - Metrics
zsdf2 = pd.read_csv(outpath)
metrics(zsdf2, corr_outpath, dump = True)

Accuracy:  62.96 %


## One Shot Validation - With Knowledge

In [ ]:
#One Shot Results - Without Knowledge
outpath = '/content/drive/MyDrive/feasibilityQA/Binary/OneShot_WK_Output.csv'
corr_outpath = '/content/drive/MyDrive/feasibilityQA/Binary/OneShot_WK_Output_Corrected.csv'

osdf1 = bdf.copy()
gpt3_response(osdf1, outpath, knowledge = True, shot = 'one', dump = True)

Streaming output truncated to the last 5000 lines.
Premise: Noah went to a shop to buy a book and gave 79 dollars to him. The shopkeeper returned some money to Noah.
Hypothesis: The cost of book could be 83.
Result: False

**********************

Row Number:  1806
Knowledge: One can not withdraw more money than the amount in their bank account.
Premise: Charlie has 622 dollars in his account. He wants to buy a bike. The cost of the vehicle is 500.
Hypothesis: Charlie could buy the vehicle from his account
Result: True

Knowledge: If the shopkeeper asks for more money then the bill would be more than the amount of money you initially gave to the shopkeeper
Premise: Noah went to a shop to buy a book and gave 79 dollars to him. The shopkeeper returned some money to Noah.
Hypothesis: The cost of book could be 76.
Result: True

**********************

Row Number:  1807
Knowledge: One can not withdraw more money than the amount in their bank account.
Premise: Charlie has 622 dollars in his a

In [ ]:
#One Shot Results - Without Knowledge (Contd...)
osdf1 = pd.read_csv(outpath)
gpt3_response(osdf1, outpath, start_idx = 2190, knowledge = True, shot = 'one', dump = True)

Streaming output truncated to the last 5000 lines.

Knowledge: Animal like horse can sleep while standing up.
Premise: In a research study on sleeping habits of animals, Leonardo came to know that there is an animal that could sleep while standing up.
Hypothesis: The animal could be a cat.
Result: False

**********************

Row Number:  2865
Knowledge: One can not withdraw more money than the amount in their bank account.
Premise: Charlie has 622 dollars in his account. He wants to buy a bike. The cost of the vehicle is 500.
Hypothesis: Charlie could buy the vehicle from his account
Result: True

Knowledge: Animals like snake doesn't have legs or hands.
Premise: Isaac saw an animal crawling in the forest. The animal had no legs and was moving on the ground.
Hypothesis: The animal could be a snake.
Result: True

**********************

Row Number:  2866
Knowledge: One can not withdraw more money than the amount in their bank account.
Premise: Charlie has 622 dollars in his account. 

In [ ]:
#Performance - Metrics
osdf1 = pd.read_csv(outpath)
metrics(osdf1, corr_outpath, dump = True)

Accuracy:  64.35 %


## One Shot Validation - Without Knowledge

In [ ]:
#One Shot Results - Without Knowledge
outpath = '/content/drive/MyDrive/feasibilityQA/Binary/OneShot_WOK_Output.csv'
corr_outpath = '/content/drive/MyDrive/feasibilityQA/Binary/OneShot_WOK_Output_Corrected.csv'

osdf2 = bdf.copy()
gpt3_response(osdf2, outpath, start_idx = 0, knowledge = False, shot = 'one', dump = True)

Streaming output truncated to the last 5000 lines.
Result: False

**********************

Row Number:  898
Premise: Charlie has 622 dollars in his account. He wants to buy a bike. The cost of the vehicle is 500.
Hypothesis: Charlie could buy the vehicle from his account
Result: True

Premise: Maxwell has two 8th grade books: Physics and Chemistry. Physics book with higher thickness has 300 pages.
Hypothesis: Number of pages in Chemistry book could be 305.
Result: False

**********************

Row Number:  899
Premise: Charlie has 622 dollars in his account. He wants to buy a bike. The cost of the vehicle is 500.
Hypothesis: Charlie could buy the vehicle from his account
Result: True

Premise: Connor has two 8th grade books: Physics and Chemistry. Physics book with higher thickness has 194 pages.
Hypothesis: Number of pages in Chemistry book could be 193.
Result: False

**********************

Row Number:  900
Premise: Charlie has 622 dollars in his account. He wants to buy a bike. The

In [ ]:
#One Shot Results - Without Knowledge (Contd...)
osdf2 = pd.read_csv(outpath)
gpt3_response(osdf2, outpath, start_idx = 1352, knowledge = False, shot = 'one', dump = True)

Streaming output truncated to the last 5000 lines.
Premise: Jordan added an item to his flower plants, since they were not blooming.
Hypothesis: He could have added gasoline.
Result: False

**********************

Row Number:  2795
Premise: Charlie has 622 dollars in his account. He wants to buy a bike. The cost of the vehicle is 500.
Hypothesis: Charlie could buy the vehicle from his account
Result: True

Premise: Luke added an item to his flower plants, since they were not blooming.
Hypothesis: He could have added fertilizer.
Result: True

**********************

Row Number:  2796
Premise: Charlie has 622 dollars in his account. He wants to buy a bike. The cost of the vehicle is 500.
Hypothesis: Charlie could buy the vehicle from his account
Result: True

Premise: Luke added an item to his flower plants, since they were not blooming.
Hypothesis: He could have added gasoline.
Result: False

**********************

Row Number:  2797
Premise: Charlie has 622 dollars in his account. He w

In [ ]:
#Performance - Metrics
metrics(osdf2, corr_outpath, dump = True)

Accuracy:  57.51 %


## Few Shot - With Knowledge

In [ ]:
#Few Shot Results - With Knowledge
outpath = '/content/drive/MyDrive/feasibilityQA/FewShot_WK_Output.csv'
corr_outpath = '/content/drive/MyDrive/feasibilityQA/FewShot_WK_Output_Corrected.csv'

fsdf1 = bdf.copy()
gpt3_response(fsdf1, outpath, knowledge = True, shot = 'few', dump = True, sample_size=0.1)

Streaming output truncated to the last 5000 lines.
Premise: Evan is a citizen of India and he recently cast his first vote in the national election.
Hypothesis: Evan is 21 years old when he casted the vote.
Result: True

Knowledge: To pass an exam, you need to get at least passing marks.
Premise: Anthony passed an exam where the passing score was 77.
Hypothesis: Anthony could have scored 76 marks in that exam.
Result: False

Knowledge: Small contact angles correspond to high wettability.
Premise: Finn is working in a car repair shop where wettability is used to prevent corrosion with two different liquids. He found that the effectiveness of two liquids is different and is verifying the change by comparing their contact angles and wettability. The contact angle of the liquid with lower wettability is 41 degrees.
Hypothesis: Contact angle of the other liquid could be 48 degrees.
Result: False

**********************

Row Number:  148
Knowledge: One can not withdraw more money than the am

In [ ]:
#Performance - Metrics
fsdf1 = pd.read_csv(outpath)
metrics(fsdf1, corr_outpath, dump = True)

Accuracy:  68.31 %


## Few Shot - Without Knowledge

In [ ]:
#Few Shot Results - Without Knowledge
outpath = '/content/drive/MyDrive/feasibilityQA/FewShot_WOK_Output.csv'
corr_outpath = '/content/drive/MyDrive/feasibilityQA/FewShot_WOK_Output_Corrected.csv'

fsdf2 = bdf.copy()
gpt3_response(fsdf2, outpath, knowledge = False, shot = 'few', dump = True, sample_size=0.1)

Streaming output truncated to the last 5000 lines.
Hypothesis: Anthony could have scored 76 marks in that exam.
Result: False

Premise: Tyler's job is in a far more colder place than where his friend stays. The temperature yesterday dropped to 121 celsius in the city where Tyler lives.
Hypothesis: The temperature where Tyler's friend lives could be 114 celsius.
Result: False

**********************

Row Number:  109
Premise: Charlie has 622 dollars in his account. He wants to buy a bike. The cost of the vehicle is 500.
Hypothesis: Charlie could buy the vehicle from his account
Result: True

Premise: Victor's experiment is to observe the relationship between resistance and area using two similar wires of different areas. The area of the wire with larger resistance is 32 units.
Hypothesis: Area of the other wire could be 25 units.
Result: False

Premise: Evan is a citizen of India and he recently cast his first vote in the national election.
Hypothesis: Evan is 21 years old when he caste

In [ ]:
#Performance - Metrics
fsdf2 = pd.read_csv(outpath)
metrics(fsdf2, corr_outpath, dump = True)

Accuracy:  64.0 %


# MCQ Question Type

## Zero Shot Validation - With Knowledge

In [ ]:
#Zero Shot Results - Without Knowledge
outpath = '/content/drive/MyDrive/feasibilityQA/MCQ/With Knowledge/ZeroShot_WK_MCQ_Output.csv'
corr_outpath = '/content/drive/MyDrive/feasibilityQA/MCQ/With Knowledge/ZeroShot_WK_MCQ_Output_Corrected.csv'

zsbf1 = mdf.copy()
gpt3_response(zsbf1, outpath, knowledge = True, shot = 'zero', dump = True, question_type = 'mcq', sample_size=0.1)

Data size:  1624
Data size after resampling:  162
Row Number:  1
Choose the correct option(s) based on Knowledge (if given) and Premise:
Choose the correct option(s) based on Knowledge (if given) and Premise:
Knowledge: Surface tension is inversely proportional to the temperature.
Premise: Liam was washing the clothes with soap and detergents and observed that the soap water did not soak well into the pores and soiled areas. So Liam changed the temperature of the water and was able to wash properly. The temperature of water in the first case is 34 units.
Question: What could be the temperature of water for the second case?
Options: [{'A': 36}, {'B': 35}, {'C': 37}, {'D': 34}, {'E': 'None'}]
Result: 'C': 37

**********************

Row Number:  2
Choose the correct option(s) based on Knowledge (if given) and Premise:
Choose the correct option(s) based on Knowledge (if given) and Premise:
Knowledge: Job of a tailor is to stitch clothes
Premise: Jameson had a family function for which he 

In [ ]:
#Performance - Metrics
zsbf1 = pd.read_csv(outpath)
metrics(zsbf1, corr_outpath, dump = True, question_type = 'mcq')

Accuracy:  44.52 %


## Zero Shot Validation - Without Knowledge

In [ ]:
#Zero Shot Results - With Knowledge
outpath = '/content/drive/MyDrive/feasibilityQA/MCQ/Without Knowledge/ZeroShot_WOK_MCQ_Output.csv'
corr_outpath = '/content/drive/MyDrive/feasibilityQA/MCQ/Without Knowledge/ZeroShot_WOK_MCQ_Output_Corrected.csv'

zsbf2 = mdf.copy()
gpt3_response(zsbf2, outpath, knowledge = False, shot = 'zero', dump = True, question_type = 'mcq', sample_size=0.1)

Data size:  1624
Data size after resampling:  162
Row Number:  1
Choose the correct option(s) based on Knowledge (if given) and Premise:
Premise: Hudson is climbing a mountain where his saturation of oxyhemogobin before start is 90 units. Hudson is facing some breathing issues.
Question: What could be the value of saturation of oxyhemoglobin?
Options: [{'A': 89}, {'B': 90}, {'C': 88}, {'D': 92}]
Result: B

**********************

Row Number:  2
Choose the correct option(s) based on Knowledge (if given) and Premise:
Premise: Adriel and his friend were living in two cities with different weather conditions such as temperature and relative humidity. The temperature at the city with lower relative humidity is 224 K.
Question: What could be the relative humidty of the other city?
Options: [{'A': 232}, {'B': 230}, {'C': 216}, {'D': 222}]
Result: C

**********************

Row Number:  3
Choose the correct option(s) based on Knowledge (if given) and Premise:
Premise: Calvin is asked to draw a

In [ ]:
#Performance - Metrics
outpath = '/content/drive/MyDrive/feasibilityQA/MCQ/Without Knowledge/ZeroShot_WOK_MCQ_Output.csv'
corr_outpath = '/content/drive/MyDrive/feasibilityQA/MCQ/Without Knowledge/ZeroShot_WOK_MCQ_Output_Corrected.csv'
zsbf2 = pd.read_csv(outpath)
metrics(zsbf2, corr_outpath, dump = True, question_type = 'mcq')

Accuracy:  47.4 %


## One Shot Validation - With Knowledge

In [ ]:
#One Shot Results - With Knowledge
outpath = '/content/drive/MyDrive/feasibilityQA/MCQ/With Knowledge/OneShot_WK_MCQ_Output.csv'
corr_outpath = '/content/drive/MyDrive/feasibilityQA/MCQ/With Knowledge/OneShot_WK_MCQ_Output_Corrected.csv'

osbf1 = mdf.copy()
gpt3_response(osbf1, outpath, knowledge = True, shot = 'one', dump = True, question_type = 'mcq', sample_size=0.1)

Data size:  1624
Data size after resampling:  162
Row Number:  1
Resources exhausted probably!!!


In [ ]:
#Performance - Metrics
outpath = '/content/drive/MyDrive/feasibilityQA/MCQ/With Knowledge/OneShot_WK_MCQ_Output.csv'
corr_outpath = '/content/drive/MyDrive/feasibilityQA/MCQ/With Knowledge/OneShot_WK_MCQ_Output_Corrected.csv'

osbf1 = pd.read_csv(outpath)
x = metrics(osbf1, corr_outpath, dump = False, question_type = 'mcq', shot = 'one')

Accuracy:  71.86 %
Accuracy2:  35.99 %
FPR:  10.51 %


In [ ]:
x.to_csv('OS.csv', index = False)

In [ ]:
# metrics(osbf1.iloc[[2]], corr_outpath, dump = False, question_type = 'mcq', shot = 'one')

In [ ]:
x.head(10)

,index,category,subcategory,premise,knowledge,question,question_text,options,value_answer,option_answer,author,GPT3Pred,TotalRes,TruePositives,TrueNegatives,GPT3Pred2List,GPT3Pred2List_Len,FalseNegatives,FalsePositives
0,1054,implicit_numerical_knowledge,decade,John's son Brandon has been in politics for 2 ...,A decade has 10 years,{'question_text': 'How many years could Brando...,How many years could Brandon have spent in pol...,"[{'A': 20}, {'B': 21}, {'C': 25}, {'D': 12}, {...","[20, 21, 25]","[A, B, C]",Neeraj,"[20, 21, 25, 12]",3,3,1,"[20, 21, 25, 12]",4,0,0
1,504,attribute_comparison,compressibility_and_density,Ian is an employee at Fire extinguisher manufa...,Compressibility factor is inversely proportion...,{'question_text': 'What could be the density o...,What could be the density of the other gas?,"[{'A': 42}, {'B': 41}, {'C': 40}, {'D': 38}]",[38],[D],Yamini,"[40, 38]",1,1,2,"[40, 38]",2,0,0
2,1368,non_numerical,tomatoes,Samuel and his brother were watching a movie i...,Animals like tiger and zebra have stripes on t...,"{'question_text': 'What could the animal be?',...",What could the animal be?,"[{'A': 'donkey'}, {'B': 'lion'}, {'C': 'tiger'...","['tiger', 'zebra']","[C, D]",Saurabh,"[tiger, zebra]",2,2,2,"[tiger, zebra]",2,0,0
3,1099,implicit_numerical_knowledge,thumb_in_hand,"In a meeting, all the attendees scanned the th...",A human usually has two hands each having one ...,{'question_text': 'How many people could have ...,How many people could have attended the meeting?,"[{'A': 10}, {'B': 19}, {'C': 20}, {'D': 24}]","[19, 20, 24]","[B, C, D]",Neeraj,"[20, 24]",3,2,2,"[20, 24]",2,0,1
4,1444,non_numerical,tomatoes,Dr.Christian is researching about fruits and t...,"Tomatoes are fruits, not vegetables.",{'question_text': 'Which fruit among these the...,Which fruit among these the lab could include ...,"[{'A': 'tomatoes'}, {'B': 'potatoes'}, {'C': '...",['tomatoes'],[A],Saurabh,['tomatoes'],1,1,3,['tomatoes'],1,0,0
5,1356,non_numerical_sg,tin_isotopes,Jonathan works in a lab on radioactive decompo...,Tin has the maximum number of stable isotopes.,{'question_text': 'What could have been the el...,What could have been the element?,"[{'A': 'Tin'}, {'B': 'Copper'}, {'C': 'Boron'}...",['Tin'],[A],Sidharth,['Tin'],1,1,3,['Tin'],1,0,0
6,795,attribute_comparison,monotonous_increase,Isaac noted the heights of all the students in...,A function is called a monotonous increase fun...,{'question_text': 'What could be the height of...,What could be the height of the student standi...,"[{'A': 113}, {'B': 105}, {'C': 106}, {'D': 112}]","[113, 112]","[A, D]",Sivapriya,"[113, 112]",2,2,2,"[113, 112]",2,0,0
7,1323,non_numerical_sg,hearing,Justin is fond of listening to music. He liste...,Hearing songs on loud volume may lead to heari...,{'question_text': 'What could have been the re...,What could have been the reason?,"[{'A': 'Eating too much'}, {'B': 'Sleeping too...",['Listening to loud music'],[D],Sidharth,[D],1,1,3,[D],1,0,0
8,1447,non_numerical,tusk,A multinational team led by the University of ...,"Animals like elephant, walrus and wild boar ha...","{'question_text': 'What could the animal be?',...",What could the animal be?,"[{'A': 'elephant'}, {'B': 'walrus'}, {'C': 'sc...","['elephant', 'walrus']","[A, B]",Saurabh,"[elephant, walrus]",2,2,2,"[elephant, walrus]",2,0,0
9,1325,non_numerical_sg,kangaroo,Kayden went on a trip to Australia. He was eag...,Kangaroo is effectively a five legged animal.,{'question_text': 'What could have been the an...,What could have been the animal that Kayden saw?,"[{'A': 'Kangaroo'}, {'B': 'Cheetah'}, {'C': 'P...",['Cheetah'],[B],Sidharth,['Kangaroo'],1,0,3,['Kangaroo'],1,0,0


## One Shot Validation - Without Knowledge

In [ ]:
#One Shot Results - Without Knowledge
outpath = '/content/drive/MyDrive/feasibilityQA/MCQ/Without Knowledge/OneShot_WOK_MCQ_Output.csv'
corr_outpath = '/content/drive/MyDrive/feasibilityQA/MCQ/Without Knowledge/OneShot_WOK_MCQ_Output_Corrected.csv'

osbf2 = mdf.copy()
gpt3_response(osbf2, outpath, knowledge = False, shot = 'one', dump = True, question_type = 'mcq', sample_size=0.2)

Data size:  1624
Data size after resampling:  325
Row Number:  1
Premise: Charlie has 622 dollars in his account. He wants to buy a bike. The cost of the vehicle is 500.
Question: At what cost could Charlie buy the vehicle from his account?
Options: [{'A': 550}, {'B': 350}, {'C': 650}, {'D': 750}, {'E': 'None'}]
Result: [550, 350]

Premise: Adam is filling his two water tank. Each tank is being filled by two identical pipes. The water tank that is filled less is being filled by a pipe with flow rate of 26 units.
Question: What could be the flow rate of other pipe ?
Hypothesis: [{'A': 23}, {'B': 25}, {'C': 22}, {'D': 29}]
Result: [25, 29]

**********************

Row Number:  2
Premise: Charlie has 622 dollars in his account. He wants to buy a bike. The cost of the vehicle is 500.
Question: At what cost could Charlie buy the vehicle from his account?
Options: [{'A': 550}, {'B': 350}, {'C': 650}, {'D': 750}, {'E': 'None'}]
Result: [550, 350]

Premise: Hayden is rowing a boat. The speed o

In [ ]:
#Performance - Metrics
outpath = '/content/drive/MyDrive/feasibilityQA/MCQ/Without Knowledge/OneShot_WOK_MCQ_Output.csv'
corr_outpath = '/content/drive/MyDrive/feasibilityQA/MCQ/Without Knowledge/OneShot_WOK_MCQ_Output_Corrected.csv'

osbf2 = pd.read_csv(outpath)
x = metrics(osbf2, corr_outpath, dump = False, question_type = 'mcq', shot = 'one')

Accuracy:  73.93 %
FPR:  7.86 %


## Few Shot - With Knowledge

In [ ]:
#Few Shot Results - Without Knowledge
outpath = '/content/drive/MyDrive/feasibilityQA/MCQ/With Knowledge/FewShot_WK_MCQ_Output.csv'
corr_outpath = '/content/drive/MyDrive/feasibilityQA/MCQ/With Knowledge/FewShot_WK_MCQ_Output_Corrected.csv'

fsbf1 = mdf.copy()
gpt3_response(fsbf1, outpath, knowledge = True, shot = 'few', dump = True, question_type = 'mcq', sample_size=0.2)

Streaming output truncated to the last 5000 lines.
Options: [{'A': 31}, {'B': 28}, {'C': 33}, {'D': 24}]
Result: [31, 33]

Knowledge: Speed of the upstream river reduces the speed of the raft.
Premise: Samuel participates in a boat racing competition. He first rows the boat in still water with a speed of 20 and then starts to go upstream.
Question: What could be the overall speed of the raft in upstream?
Options: [{'A': 24}, {'B': 15}, {'C': 16}, {'D': 19}]
Result: [15, 16, 19]

Knowledge: A region's population grows either by birth or immigration.
Premise: It is observed that there has been an increase in people at Tempe recently.
Question: Wha could be reason for increase in population?
Hypothesis: [{'A': 'divorce'}, {'B': 'consumption of food'}, {'C': 'death'}, {'D': 'immigration'}]
Result: [ immigration]

**********************

Row Number:  175
Knowledge: One can not withdraw more money than the amount in their bank account.
Premise: Charlie has 622 dollars in his account. He want

In [ ]:
#Performance - Metrics
outpath = '/content/drive/MyDrive/feasibilityQA/MCQ/With Knowledge/FewShot_WK_MCQ_Output.csv'
corr_outpath = '/content/drive/MyDrive/feasibilityQA/MCQ/With Knowledge/FewShot_WK_MCQ_Output_Corrected.csv'

fsbf1 = pd.read_csv(outpath)
x = metrics(fsbf1, corr_outpath, dump = False, question_type = 'mcq', shot = 'few')

Accuracy:  74.39 %
FPR:  8.82 %


## Few Shot - Without Knowledge

In [ ]:
#One Shot Results - Without Knowledge
outpath = '/content/drive/MyDrive/feasibilityQA/MCQ/Without Knowledge/FewShot_WOK_MCQ_Output.csv'
corr_outpath = '/content/drive/MyDrive/feasibilityQA/MCQ/Without Knowledge/FewShot_WOK_MCQ_Output_Corrected.csv'

fsbf2 = mdf.copy()
gpt3_response(fsbf2, outpath, knowledge = False, shot = 'few', dump = True, question_type = 'mcq', sample_size=0.2)

Streaming output truncated to the last 5000 lines.
Question: What could be the potential energy of the other ball?
Options: [{'A': 31}, {'B': 28}, {'C': 33}, {'D': 24}]
Result: [31, 33]

Premise: Samuel participates in a boat racing competition. He first rows the boat in still water with a speed of 20 and then starts to go upstream.
Question: What could be the overall speed of the raft in upstream?
Options: [{'A': 24}, {'B': 15}, {'C': 16}, {'D': 19}]
Result: [15, 16, 19]

Premise: A truck and a bike were travelling on a straight road. Truck was moving faster than bike with 70 miles/hour.
Question: What could be the speed of the bike ?
Hypothesis: [{'A': 74}, {'B': 69}, {'C': 71}, {'D': 73}, {'E': 'None'}]
Result: [69, 71, 73]

**********************

Row Number:  148
Premise: Charlie has 622 dollars in his account. He wants to buy a bike. The cost of the vehicle is 500.
Question: At what cost could Charlie buy the vehicle from his account?
Options: [{'A': 550}, {'B': 350}, {'C': 650},

In [ ]:
#Performance - Metrics
outpath = '/content/drive/MyDrive/feasibilityQA/MCQ/Without Knowledge/FewShot_WOK_MCQ_Output.csv'
corr_outpath = '/content/drive/MyDrive/feasibilityQA/MCQ/Without Knowledge/FewShot_WOK_MCQ_Output_Corrected.csv'

fsbf2 = pd.read_csv(outpath)
x = metrics(fsbf2, corr_outpath, dump = False, question_type = 'mcq', shot = 'few')

Accuracy:  75.61 %
FPR:  6.67 %
